## Data 690 (AI) Final Project
## Facial Recognition Demo
## Yunpeng Li

In [1]:
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

## Function for creating the directories for storing images:

In [2]:
def makedir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
        pass


## Function for extracting person faces from original photo in a source directory and save them to a destination directory:

In [3]:
def extractFacesFromPictures(source_dir, dest_dir):
    
    makedir(dest_dir)
    
    # Loading HAARCascade Face Detector 
    faceDetector = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
    
    imageNames = [f for f in listdir(source_dir) if isfile(join(source_dir, f))]
    
    for imageName in imageNames:
        print(imageName)
        image = cv2.imread(source_dir+imageName)
        
        cv2.imshow('imageName', image) 
        cv2.waitKey(0)
        
        face_info = faceDetector.detectMultiScale(image, 1.3, 5)
        
        print(face_info)
        for (x,y,w,h) in face_info:
            face = image[y:y+h, x:x+w]
            roi = cv2.resize(face, (128, 128), interpolation = cv2.INTER_CUBIC)
        
            cv2.imshow("face", roi)
            
            path = dest_dir + "face_" + imageName 
            cv2.imwrite(path, roi) #Save the extracted faces into the destination directory
            
            cv2.waitKey(0)
        
    cv2.destroyAllWindows()

## Use VGGFaceModel

In [4]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

model.load_weights('vgg_face_weights.h5')

#Remove last Softmax layer and get model upto last flatten layer
vggFaceModel = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

## VggFaceModel architecture

In [5]:
vggFaceModel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_input (InputL [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0     

## Function for loading image from an image path and resizes it:

In [6]:
def preprocess_image(image_path):
    """Loads image from path and resizes it"""
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

## Function for comparing images by Cosine Similiarity (the lower score, the more similiar):

In [7]:
def calculateCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

## Extract faces from photos in images folder and save them in faces folder, which is to set up our very simple facial database.  "Enter" key needs to be pressed to process each photo: 

In [8]:
extractFacesFromPictures('./images/', './faces/')

Britney Spears.jpg
[[403 307 830 830]]
Dwayne Johnson (Rock).jpg
[[ 67  63 185 185]]
Hillary.jpg
[[392 612 836 836]]
Kim Jong Un.jpg
[[ 80  74 189 189]]
Lady Gaga.jpg
[[155  53 413 413]]
Madonna.jpg
[[507 171 516 516]]
Trump.jpg
[[1448  457  890  890]]


## Create the presentation of each extraced face by VGGFaceModel prediction.  The facial representation is a vector or array, and the associated key is the person's name prefixed with "face_" :

In [9]:
def create_face_vector_representation_dictionary(faces_repository):
    face_dict = dict()

    for file in listdir(faces_repository):
        print('Creating facial representation for', file)
        person_face, extension = file.split(".")
        face_dict[person_face] = vggFaceModel.predict(preprocess_image('./faces/%s.jpg' % (person_face)))[0,:]
    return face_dict

In [10]:
face_db_dictionary = create_face_vector_representation_dictionary('./faces/')

Creating facial representation for face_Britney Spears.jpg
Creating facial representation for face_Dwayne Johnson (Rock).jpg
Creating facial representation for face_Hillary.jpg
Creating facial representation for face_Kim Jong Un.jpg
Creating facial representation for face_Lady Gaga.jpg
Creating facial representation for face_Madonna.jpg
Creating facial representation for face_Trump.jpg


In [11]:
face_db_dictionary

{'face_Britney Spears': array([ 1.60496   , -1.5575426 , -0.14216928, ...,  3.4180443 ,
        -0.42121536,  0.14598373], dtype=float32),
 'face_Dwayne Johnson (Rock)': array([ 3.9746056 , -2.6902373 , -0.30208504, ..., -1.9033624 ,
         0.47788623,  0.14036633], dtype=float32),
 'face_Hillary': array([ 1.0320519, -2.672826 , -3.1798182, ..., -2.312236 ,  0.9663745,
         1.2726313], dtype=float32),
 'face_Kim Jong Un': array([ 0.81118304, -1.084969  ,  0.1463707 , ..., -2.7813516 ,
         4.306972  ,  0.53672796], dtype=float32),
 'face_Lady Gaga': array([ 1.7031907 , -4.33134   , -2.6338637 , ..., -1.8551259 ,
         0.06040401,  3.3642864 ], dtype=float32),
 'face_Madonna': array([ 0.68289185, -0.9796021 ,  1.4411439 , ..., -4.76454   ,
         2.0778015 ,  2.0321078 ], dtype=float32),
 'face_Trump': array([ 2.4377742 , -0.18090715, -1.6431191 , ..., -1.5195199 ,
        -1.0417398 , -0.7483742 ], dtype=float32)}

## Function for detecting and recognizing faces in a given image:

In [12]:
def recognize_face_from_image(img, face_database):
    
    # Loading HAARCascade Face Detector 
    faceDetector = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

    faces = faceDetector.detectMultiScale(img, 1.3, 5)

    for (x,y,w,h) in faces:

        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2) #draw rectangle to main image

        detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
        detected_face = cv2.resize(detected_face, (224, 224)) #resize to 224x224

        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        captured_representation = vggFaceModel.predict(img_pixels)[0,:]

        #print(captured_representation)

        found = False

        # Compare captured face with the stored face
        for face in face_database:
            representation = face_database[face]
            person_name=face[5:]

            similarity = calculateCosineSimilarity(representation, captured_representation)
            
            print('Cosine Similarity score by comparing with face of ' + person_name + ':' + str(similarity))

            if(similarity < 0.25):
                cv2.putText(img, person_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                found = True
                break

        #connect face and text

        cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255, 0, 0),1)
        cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255, 0, 0),1)

        if(found == False): #if found image is not in our people database
            cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


        cv2.imshow('img',img)


## Function for using cv2.VideoCapture to recognize face similar to the face images stored in the face_database:

In [13]:
def recognize_face(face_database, video_capture):
    
    while(True):
        ret, img = video_capture.read()
        
        recognize_face_from_image(img, face_database)

        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break

    video_capture.release()
    cv2.destroyAllWindows()

## Detect and recognize faces in an image:

In [14]:
test_img_file = './test_images/lady_gaga.jpg'

test_image = cv2.imread(test_img_file)

In [15]:
recognize_face_from_image(test_image, face_db_dictionary)

key = cv2.waitKey(0) & 0xFF
if key == 13:
    cv2.destroyAllWindows()

Cosine Similarity score by comparing with face of Britney Spears:0.6327725648880005
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.6813664138317108
Cosine Similarity score by comparing with face of Hillary:0.6192145943641663
Cosine Similarity score by comparing with face of Kim Jong Un:0.6973978281021118
Cosine Similarity score by comparing with face of Lady Gaga:0.2482903003692627


In [16]:
cv2.destroyAllWindows()

### Use webcam to detect and capture face in real time and compare it with the stored faces so as to recogize face.  Note: You need to first put a copy of your own facial image in the images folder and go through the previous steps in order to enable the following code to recogize your face captured by the camera.  Press "Enter" key to stop:  

In [17]:
#Open Webcam
camera = cv2.VideoCapture(0)

recognize_face(face_db_dictionary, camera)

camera.release()

Cosine Similarity score by comparing with face of Britney Spears:0.7083582878112793
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5186364650726318
Cosine Similarity score by comparing with face of Hillary:0.5239353775978088
Cosine Similarity score by comparing with face of Kim Jong Un:0.3824232816696167
Cosine Similarity score by comparing with face of Lady Gaga:0.6857924163341522
Cosine Similarity score by comparing with face of Madonna:0.7964036762714386
Cosine Similarity score by comparing with face of Trump:0.3654879927635193
Cosine Similarity score by comparing with face of Britney Spears:0.6690274178981781
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5272611081600189
Cosine Similarity score by comparing with face of Hillary:0.5163277387619019
Cosine Similarity score by comparing with face of Kim Jong Un:0.4429488182067871
Cosine Similarity score by comparing with face of Lady Gaga:0.6617435216903687
Cosine Similarity score 

### Analyze recorded video to recognize faces in the video.  Press "Enter" key to stop:  

In [18]:
video_capture_1 = cv2.VideoCapture('./videos/1.mp4')

recognize_face(face_db_dictionary, video_capture_1)

video_capture_1.release()

Cosine Similarity score by comparing with face of Britney Spears:0.5069708526134491
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.6467167735099792
Cosine Similarity score by comparing with face of Hillary:0.44458556175231934
Cosine Similarity score by comparing with face of Kim Jong Un:0.4134952425956726
Cosine Similarity score by comparing with face of Lady Gaga:0.6124760508537292
Cosine Similarity score by comparing with face of Madonna:0.7012667655944824
Cosine Similarity score by comparing with face of Trump:0.17816162109375
Cosine Similarity score by comparing with face of Britney Spears:0.4899384379386902
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.6442877054214478
Cosine Similarity score by comparing with face of Hillary:0.43871772289276123
Cosine Similarity score by comparing with face of Kim Jong Un:0.4212968349456787
Cosine Similarity score by comparing with face of Lady Gaga:0.5869112312793732
Cosine Similarity score 

In [19]:
video_capture_2 = cv2.VideoCapture('./videos/2.mp4')

recognize_face(face_db_dictionary, video_capture_2)

video_capture_2.release()

Cosine Similarity score by comparing with face of Britney Spears:0.7185327708721161
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5784184038639069
Cosine Similarity score by comparing with face of Hillary:0.5445406436920166
Cosine Similarity score by comparing with face of Kim Jong Un:0.09662479162216187
Cosine Similarity score by comparing with face of Britney Spears:0.6176142394542694
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5833520889282227
Cosine Similarity score by comparing with face of Hillary:0.463603675365448
Cosine Similarity score by comparing with face of Kim Jong Un:0.34067171812057495
Cosine Similarity score by comparing with face of Lady Gaga:0.6697556674480438
Cosine Similarity score by comparing with face of Madonna:0.8119717836380005
Cosine Similarity score by comparing with face of Trump:0.12252944707870483
Cosine Similarity score by comparing with face of Britney Spears:0.7045881152153015
Cosine Similarity

Cosine Similarity score by comparing with face of Britney Spears:0.7043907940387726
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5832531750202179
Cosine Similarity score by comparing with face of Hillary:0.5271687805652618
Cosine Similarity score by comparing with face of Kim Jong Un:0.12274503707885742
Cosine Similarity score by comparing with face of Britney Spears:0.5760060846805573
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5692760944366455
Cosine Similarity score by comparing with face of Hillary:0.48956137895584106
Cosine Similarity score by comparing with face of Kim Jong Un:0.3611098527908325
Cosine Similarity score by comparing with face of Lady Gaga:0.6529722809791565
Cosine Similarity score by comparing with face of Madonna:0.8272641152143478
Cosine Similarity score by comparing with face of Trump:0.10821032524108887


In [46]:
video_capture_3 = cv2.VideoCapture('./videos/3.mp4')

recognize_face(face_db_dictionary, video_capture_3)

video_capture_3.release()

Cosine Similarity score by comparing with face of Britney Spears:0.7185327708721161
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5784184038639069
Cosine Similarity score by comparing with face of Hillary:0.5445406436920166
Cosine Similarity score by comparing with face of Jenny:0.36314815282821655
Cosine Similarity score by comparing with face of Kim Jong Un:0.09662479162216187
Cosine Similarity score by comparing with face of Britney Spears:0.6176142394542694
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5833520889282227
Cosine Similarity score by comparing with face of Hillary:0.463603675365448
Cosine Similarity score by comparing with face of Jenny:0.39346396923065186
Cosine Similarity score by comparing with face of Kim Jong Un:0.34067171812057495
Cosine Similarity score by comparing with face of Lady Gaga:0.6697556674480438
Cosine Similarity score by comparing with face of Madonna:0.8119717836380005
Cosine Similarity score b

Cosine Similarity score by comparing with face of Britney Spears:0.6871045827865601
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.6586930453777313
Cosine Similarity score by comparing with face of Hillary:0.6758392453193665
Cosine Similarity score by comparing with face of Jenny:0.5225062072277069
Cosine Similarity score by comparing with face of Kim Jong Un:0.6531893908977509
Cosine Similarity score by comparing with face of Lady Gaga:0.6655979454517365
Cosine Similarity score by comparing with face of Madonna:0.7613242566585541
Cosine Similarity score by comparing with face of Trump:0.5818934440612793
Cosine Similarity score by comparing with face of Yunpeng:0.5325725674629211
Cosine Similarity score by comparing with face of Britney Spears:0.5762434601783752
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5868694484233856
Cosine Similarity score by comparing with face of Hillary:0.5001464486122131
Cosine Similarity score by compa

Cosine Similarity score by comparing with face of Britney Spears:0.7103865742683411
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5814467668533325
Cosine Similarity score by comparing with face of Hillary:0.5264227986335754
Cosine Similarity score by comparing with face of Jenny:0.3459824323654175
Cosine Similarity score by comparing with face of Kim Jong Un:0.10087096691131592
Cosine Similarity score by comparing with face of Britney Spears:0.5432164072990417
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5809255242347717
Cosine Similarity score by comparing with face of Hillary:0.4682652950286865
Cosine Similarity score by comparing with face of Jenny:0.4237927794456482
Cosine Similarity score by comparing with face of Kim Jong Un:0.3642734885215759
Cosine Similarity score by comparing with face of Lady Gaga:0.6333576142787933
Cosine Similarity score by comparing with face of Madonna:0.8014983087778091
Cosine Similarity score by 

Cosine Similarity score by comparing with face of Britney Spears:0.7042480409145355
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5702287554740906
Cosine Similarity score by comparing with face of Hillary:0.5231492519378662
Cosine Similarity score by comparing with face of Jenny:0.3574896454811096
Cosine Similarity score by comparing with face of Kim Jong Un:0.1101275086402893
Cosine Similarity score by comparing with face of Britney Spears:0.5412896275520325
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5487488508224487
Cosine Similarity score by comparing with face of Hillary:0.4909765124320984
Cosine Similarity score by comparing with face of Jenny:0.48629122972488403
Cosine Similarity score by comparing with face of Kim Jong Un:0.36554503440856934
Cosine Similarity score by comparing with face of Lady Gaga:0.6571992039680481
Cosine Similarity score by comparing with face of Madonna:0.8272671848535538
Cosine Similarity score by

Cosine Similarity score by comparing with face of Britney Spears:0.6877558529376984
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.6472401022911072
Cosine Similarity score by comparing with face of Hillary:0.6834765374660492
Cosine Similarity score by comparing with face of Jenny:0.5194221138954163
Cosine Similarity score by comparing with face of Kim Jong Un:0.6498687863349915
Cosine Similarity score by comparing with face of Lady Gaga:0.6766581237316132
Cosine Similarity score by comparing with face of Madonna:0.7640417814254761
Cosine Similarity score by comparing with face of Trump:0.5848886668682098
Cosine Similarity score by comparing with face of Yunpeng:0.5313048362731934
Cosine Similarity score by comparing with face of Britney Spears:0.5515630543231964
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5646861791610718
Cosine Similarity score by comparing with face of Hillary:0.46247971057891846
Cosine Similarity score by comp

...

Cosine Similarity score by comparing with face of Britney Spears:0.5977509021759033
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.585017591714859
Cosine Similarity score by comparing with face of Hillary:0.43424874544143677
Cosine Similarity score by comparing with face of Jenny:0.44470465183258057
Cosine Similarity score by comparing with face of Kim Jong Un:0.40048420429229736
Cosine Similarity score by comparing with face of Lady Gaga:0.6274421811103821
Cosine Similarity score by comparing with face of Madonna:0.8329505920410156
Cosine Similarity score by comparing with face of Trump:0.15331894159317017
Cosine Similarity score by comparing with face of Britney Spears:0.6147045791149139
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5511384308338165
Cosine Similarity score by comparing with face of Hillary:0.42394810914993286
Cosine Similarity score by comparing with face of Jenny:0.39400482177734375
Cosine Similarity score by co

Cosine Similarity score by comparing with face of Britney Spears:0.7058532536029816
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5804698467254639
Cosine Similarity score by comparing with face of Hillary:0.5323531925678253
Cosine Similarity score by comparing with face of Jenny:0.39149099588394165
Cosine Similarity score by comparing with face of Kim Jong Un:0.1364375352859497
Cosine Similarity score by comparing with face of Britney Spears:0.6597359776496887
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5421823561191559
Cosine Similarity score by comparing with face of Hillary:0.45166468620300293
Cosine Similarity score by comparing with face of Jenny:0.35738635063171387
Cosine Similarity score by comparing with face of Kim Jong Un:0.3888087868690491
Cosine Similarity score by comparing with face of Lady Gaga:0.6279438436031342
Cosine Similarity score by comparing with face of Madonna:0.8015540242195129
Cosine Similarity score b

Cosine Similarity score by comparing with face of Britney Spears:0.5995081067085266
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5685475766658783
Cosine Similarity score by comparing with face of Hillary:0.42070096731185913
Cosine Similarity score by comparing with face of Jenny:0.40663355588912964
Cosine Similarity score by comparing with face of Kim Jong Un:0.3812703490257263
Cosine Similarity score by comparing with face of Lady Gaga:0.675476998090744
Cosine Similarity score by comparing with face of Madonna:0.8315762728452682
Cosine Similarity score by comparing with face of Trump:0.11375528573989868
Cosine Similarity score by comparing with face of Britney Spears:0.7034233808517456
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.575062483549118
Cosine Similarity score by comparing with face of Hillary:0.5341249108314514
Cosine Similarity score by comparing with face of Jenny:0.37279701232910156
Cosine Similarity score by compa

Cosine Similarity score by comparing with face of Britney Spears:0.5912195146083832
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5699493587017059
Cosine Similarity score by comparing with face of Hillary:0.43771034479141235
Cosine Similarity score by comparing with face of Jenny:0.444571852684021
Cosine Similarity score by comparing with face of Kim Jong Un:0.4010421633720398
Cosine Similarity score by comparing with face of Lady Gaga:0.6769591569900513
Cosine Similarity score by comparing with face of Madonna:0.8228899836540222
Cosine Similarity score by comparing with face of Trump:0.14927339553833008
Cosine Similarity score by comparing with face of Britney Spears:0.7082943320274353
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.569694310426712
Cosine Similarity score by comparing with face of Hillary:0.5546618700027466
Cosine Similarity score by comparing with face of Jenny:0.3810579180717468
Cosine Similarity score by compari

Cosine Similarity score by comparing with face of Britney Spears:0.5669673085212708
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5567036867141724
Cosine Similarity score by comparing with face of Hillary:0.45019763708114624
Cosine Similarity score by comparing with face of Jenny:0.41629254817962646
Cosine Similarity score by comparing with face of Kim Jong Un:0.4104388356208801
Cosine Similarity score by comparing with face of Lady Gaga:0.654239147901535
Cosine Similarity score by comparing with face of Madonna:0.8105512261390686
Cosine Similarity score by comparing with face of Trump:0.1507624387741089
Cosine Similarity score by comparing with face of Britney Spears:0.7121701836585999
Cosine Similarity score by comparing with face of Dwayne Johnson (Rock):0.5818490386009216
Cosine Similarity score by comparing with face of Hillary:0.561607837677002
Cosine Similarity score by comparing with face of Jenny:0.40028882026672363
Cosine Similarity score by compar

## Release camera or video caputure in case they are not released due to errors:

In [47]:
camera.release()
video_capture_1.release()
video_capture_2.release()
video_capture_3.release()

In [48]:
cv2.destroyAllWindows()

## Reference:

Some code is borrowed from (Hands-On-Computer-Vision-with-OpenCV-4-Keras-and-TensorFlow-2 of PacktPublishing):

https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-OpenCV-4-Keras-and-TensorFlow-2/tree/master/Section%205

Download pretrained weights (vgg_face_weights.h5) from:

https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing